<a href="https://colab.research.google.com/github/xmks-colab/SmartCutter/blob/main/smartcutter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Algumas informaçõe { display-mode: "form" }
from IPython.display import display, HTML

html_content = """
<div style='background-color: #e7f3ff; padding: 15px; border-radius: 8px; margin: 15px 0; color: #000000; font-family: Arial, sans-serif;'>
  <h3 style='color: #1a73e8; margin-bottom: 10px;'></h3>

  <h4 style='color: #202124; margin-top: 20px; margin-bottom: 10px;'>Modelo Treinado</h4>
  <p style='margin-bottom: 10px;'><strong>Descrição:</strong> Detecta bem diálogos, mas é péssimo para onomatopeias. É mais demorado, em compensação o tamanho de cada corte é bem próximo do especificado, gerando menos pedaços.</p>
  <ul style='list-style-type: disc; margin-left: 20px;'>
    <li><strong>LIMITE_MERGE_ULTIMO:</strong> Se o último pedaço for menor que x, junta com a imagem anterior. Assim, evita pedaços muito pequenos.</li>
    <li><strong>MARGEM_CORTE:</strong> Essa é uma margem de segurança. Quando é encontrada uma área segura para o corte, adiciona a margem de segurança em px para o corte não ser muito próximo do diálogo.</li>
  </ul>

  <h4 style='color: #202124; margin-top: 20px; margin-bottom: 10px;'>SmartStitch</h4>
  <p style='margin-bottom: 10px;'><strong>Descrição:</strong> Uma solução muito rápida e inteligente. Boa para pular onomatopeias e diálogos. Os tamanhos são mais variados, gerando mais cortes.</p>
  <ul style='list-style-type: disc; margin-left: 20px;'>
    <li><strong>Limpar Entrada/Saída:</strong> Limpa a pasta de upload/saída. Evita mistura se usar mais de uma vez.</li>
  </ul>
  <p style='margin-top: 10px;'>Para as demais configurações do SmartStitch:
    <a href="https://github.com/MechTechnology/SmartStitch" target="_blank" style="color: #1a73e8; text-decoration: none; font-weight: bold;">GitHub</a>
  </p>
</div>
"""

display(HTML(html_content))

In [ ]:

#@title Modelo treinado { display-mode: "form" }
import os
import glob
import numpy as np
import cv2
from PIL import Image
from pathlib import Path
import urllib.request
import shutil
from time import time
from google.colab import files
%cd /content

# ===== CONFIGURAÇÕES =====
ALTURA_CORTE = 4000           #@param {"type":"number"}
THRESHOLD_TEXTO = 0.5      #@param {"type":"number"}
MARGEM_CORTE = 50             #@param {"type":"number"}
# se a imagem final for menor que que x, ele junta com a imagem anterio. evitando pedaços muito pequenos.
LIMITE_MERGE_ULTIMO = 1000    #@param {"type":"number"}
PROCESSAR_TODAS = True
ZIPAR_RESULTADO = True   # True para gerar ZIP no final
PASTA_ORIGEM = "uploads" # Pasta com as imagens originais
PASTA_DESTINO = "cortes" # Pasta onde serão salvos os cortes
FORMATO_SAIDA = "JPG"    #@param ["JPG", "PNG"]
QUALIDADE_JPG = 95       #@param {type:"slider", min:0, max:100}
# =========================



base_dir = "/content"
uploads_dir = os.path.join(base_dir, "uploads")
cortes_dir = os.path.join(base_dir, "cortes")

arquivos_uploads = []
if os.path.exists(uploads_dir):
    arquivos_uploads = [f for f in os.listdir(uploads_dir) if os.path.isfile(os.path.join(uploads_dir, f))]

if os.path.exists(uploads_dir):
    for f in os.listdir(uploads_dir):
        caminho = os.path.join(uploads_dir, f)
        if os.path.isfile(caminho):
            os.remove(caminho)
        elif os.path.isdir(caminho):
            shutil.rmtree(caminho)

if os.path.exists(cortes_dir):
    for f in os.listdir(cortes_dir):
        caminho = os.path.join(cortes_dir, f)
        if os.path.isfile(caminho):
            os.remove(caminho)
        elif os.path.isdir(caminho):
            shutil.rmtree(caminho)

for f in arquivos_uploads:
    caminho = os.path.join(base_dir, f)
    if os.path.isfile(caminho):
        os.remove(caminho)



def resetar_pastas():
    for pasta in [PASTA_ORIGEM, PASTA_DESTINO]:
        if os.path.exists(pasta):
            shutil.rmtree(pasta)
        os.makedirs(pasta)

def fazer_upload_imagens():
    print("📤 Por favor, faça o upload das imagens agora.")
    uploaded = files.upload()
    for nome_arquivo, conteudo in uploaded.items():
        caminho_destino = os.path.join(PASTA_ORIGEM, nome_arquivo)
        with open(caminho_destino, 'wb') as f:
            f.write(conteudo)
    print(f"✅ {len(uploaded)} imagens uploadadas para {PASTA_ORIGEM}.")

def baixar_modelo_east():
    """Baixa o modelo EAST se não existir."""
    modelo_path = "east.pb"
    if not os.path.exists(modelo_path):
        print("📥 Baixando modelo EAST...")
        url = "https://github.com/oyyd/frozen_east_text_detection.pb/raw/master/frozen_east_text_detection.pb"
        urllib.request.urlretrieve(url, modelo_path)
        print("✅ Modelo baixado!")
    return modelo_path

def detectar_texto_regioes(imagem_path, modelo_east_path):
    """Detecta regiões com texto na imagem e retorna lista de (y1, y2)."""
    imagem = cv2.imread(imagem_path)
    if imagem is None:
        return []

    altura_orig, largura_orig = imagem.shape[:2]
    nova_altura = (altura_orig // 32) * 32
    nova_largura = (largura_orig // 32) * 32
    if nova_altura == 0 or nova_largura == 0:
        return []

    imagem_redim = cv2.resize(imagem, (nova_largura, nova_altura))
    blob = cv2.dnn.blobFromImage(imagem_redim, 1.0, (nova_largura, nova_altura),
                                 (123.68, 116.78, 103.94), swapRB=True, crop=False)

    net = cv2.dnn.readNet(modelo_east_path)
    net.setInput(blob)
    (scores, geometry) = net.forward(["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"])

    regioes_texto = []
    altura_feature, largura_feature = scores.shape[2:4]
    for y in range(altura_feature):
        scores_linha = scores[0, 0, y]
        geo_linha = geometry[0, :, y]
        for x in range(largura_feature):
            if scores_linha[x] < THRESHOLD_TEXTO:
                continue
            h = geo_linha[0, x] + geo_linha[2, x]
            w = geo_linha[1, x] + geo_linha[3, x]
            offset_x, offset_y = x * 4.0, y * 4.0
            x1 = int((offset_x - w / 2) * largura_orig / nova_largura)
            y1 = int((offset_y - h / 2) * altura_orig / nova_altura)
            x2 = int((offset_x + w / 2) * largura_orig / nova_largura)
            y2 = int((offset_y + h / 2) * altura_orig / nova_altura)
            regioes_texto.append((y1, y2))
    return sorted(regioes_texto, key=lambda x: x[0])

def encontrar_melhor_ponto_corte(altura_total, ponto_desejado, regioes_texto):
    ponto = ponto_desejado
    while True:
        conflito = False
        for y1, y2 in regioes_texto:
            if y1 <= ponto <= y2:
                conflito = True
                if y2 + MARGEM_CORTE < altura_total:
                    ponto = y2 + MARGEM_CORTE
                elif y1 - MARGEM_CORTE > 0:
                    ponto = y1 - MARGEM_CORTE
                break
        if not conflito:
            break
    return min(ponto, altura_total)

def obter_info_imagens(imagens):
    info_imagens = []
    y_acumulado = 0
    largura_max = 0
    for caminho in imagens:
        with Image.open(caminho) as img:
            largura, altura = img.size
            info_imagens.append({
                'caminho': caminho,
                'altura': altura,
                'largura': largura,
                'y_inicio': y_acumulado,
                'y_fim': y_acumulado + altura
            })
            y_acumulado += altura
            largura_max = max(largura_max, largura)
    return info_imagens, y_acumulado, largura_max

def mapear_regioes_texto(imagens, modelo_east):
    regioes_texto_global = []
    for info in imagens:
        regioes = detectar_texto_regioes(info['caminho'], modelo_east)
        for y1, y2 in regioes:
            regioes_texto_global.append((y1 + info['y_inicio'], y2 + info['y_inicio']))
    return sorted(regioes_texto_global, key=lambda x: x[0])

def calcular_pontos_corte(altura_total, regioes_texto):
    """Calcula pontos de corte no espaço global."""
    pontos_corte = []
    y_atual = 0
    while y_atual < altura_total:
        proximo_y = min(y_atual + ALTURA_CORTE, altura_total)
        if regioes_texto and proximo_y < altura_total:
            regioes_relevantes = [(y1, y2) for y1, y2 in regioes_texto
                                  if abs(y1 - proximo_y) <= 200 or abs(y2 - proximo_y) <= 200]
            if regioes_relevantes:
                proximo_y = encontrar_melhor_ponto_corte(altura_total, proximo_y, regioes_relevantes)
        pontos_corte.append((y_atual, proximo_y))
        y_atual = proximo_y
    return pontos_corte

def cortar_imagens_globais(info_imagens, pontos_corte, largura_max, pasta_destino):
    """Corta as imagens com base nos pontos de corte globais."""
    num_pedaco = 1
    ext = "png" if FORMATO_SAIDA == "PNG" else "jpg"
    for y_inicio_global, y_fim_global in pontos_corte:
        # Cria uma nova imagem para o pedaço
        altura_pedaco = y_fim_global - y_inicio_global
        pedaco = Image.new('RGB', (largura_max, altura_pedaco), (255, 255, 255))

        y_offset = 0
        for info in info_imagens:
            y_inicio_img = info['y_inicio']
            y_fim_img = info['y_fim']
            caminho = info['caminho']
            largura_img = info['largura']

            if y_inicio_global < y_fim_img and y_fim_global > y_inicio_img:
                with Image.open(caminho) as img:

                    y_inicio_local = max(0, y_inicio_global - y_inicio_img)
                    y_fim_local = min(info['altura'], y_fim_global - y_inicio_img)


                    area_corte = (0, y_inicio_local, largura_img, y_fim_local)
                    parte_img = img.crop(area_corte)


                    pedaco.paste(parte_img, (0, y_offset))
                    y_offset += (y_fim_local - y_inicio_local)


        nome_pedaco = f"{num_pedaco:02d}.{ext}"
        caminho_salvar = os.path.join(pasta_destino, nome_pedaco)
        if FORMATO_SAIDA == "JPG":
            pedaco.save(caminho_salvar, quality=QUALIDADE_JPG)
        else:
            pedaco.save(caminho_salvar)
        print(f"✅ Pedaço {nome_pedaco} ({largura_max}x{altura_pedaco}px) salvo")
        num_pedaco += 1
    return num_pedaco

def main():

    resetar_pastas()


    fazer_upload_imagens()

    modelo_east = baixar_modelo_east()


    imagens = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp']:
        imagens.extend(glob.glob(os.path.join(PASTA_ORIGEM, ext)))
    imagens.sort()

    if not imagens:
        print("❌ Nenhuma imagem encontrada!")
        return

    print(f"📸 {len(imagens)} imagens encontradas.")
    inicio = time()


    info_imagens, altura_total, largura_max = obter_info_imagens(imagens)
    print(f"📏 Altura total do painel virtual: {altura_total}px")
    print(f"📏 Largura máxima: {largura_max}px")


    print("🔍 Detectando texto em todas as imagens...")
    regioes_texto = mapear_regioes_texto(info_imagens, modelo_east)
    print(f"📝 {len(regioes_texto)} região(ões) de texto detectada(s)")


    print("🎯 Calculando pontos de corte...")
    pontos_corte = calcular_pontos_corte(altura_total, regioes_texto)


    if LIMITE_MERGE_ULTIMO > 0 and len(pontos_corte) > 1:
        last_height = pontos_corte[-1][1] - pontos_corte[-1][0]
        if last_height < LIMITE_MERGE_ULTIMO:
            pontos_corte[-2] = (pontos_corte[-2][0], pontos_corte[-1][1])
            del pontos_corte[-1]
            print("🔄 Último pedaço pequeno mesclado com o anterior.")

    print(f"✂️ {len(pontos_corte)} pedaços serão gerados")


    print("🔪 Cortando imagens...")
    num_pedacos = cortar_imagens_globais(info_imagens, pontos_corte, largura_max, PASTA_DESTINO)

    duracao = time() - inicio
    print(f"✅ Corte concluído em {duracao:.2f}s")
    print(f"🎉 Total de {num_pedacos-1} pedaços gerados")

    if ZIPAR_RESULTADO:
        shutil.make_archive("cortes", 'zip', PASTA_DESTINO)
        print("📦 Arquivo cortes.zip criado!")
        zip_path = '/content/cortes.zip'
        files.download(zip_path)

if __name__ == "__main__":
    main()

In [ ]:

#@title SmartStitch { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
from google.colab import files
import shutil
import sys

def run_command(cmd, silent=True):
    """Executa comando silenciosamente"""
    try:
        if silent:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
        else:
            result = subprocess.run(cmd, shell=True, check=True)
        return True, result
    except subprocess.CalledProcessError as e:
        return False, e

def install_smartstitch():
    print("Preparando...")

    # Verifica se o diretório já existe
    if os.path.exists('SmartStitch'):
        print("SmartStitch já está instalado. Ignorando...")
        os.chdir('SmartStitch')
        print("Finalizado.")
        return

    # Clone do repositório
    success, _ = run_command("git clone https://github.com/MechTechnology/SmartStitch.git")
    if not success:
        print("Erro ao clonar o repositório.")
        return

    # Mudança para o diretório
    os.chdir('SmartStitch')

    # Instalação das dependências
    packages = ["numpy", "pillow", "natsort", "psd-tools"]
    for package in packages:
        success, _ = run_command(f"pip install {package}")
        if not success:
            print(f"Erro ao instalar {package}.")
            return

    print("Finalizado.")

# Executar instalação
install_smartstitch()

# --- Criar pasta input automaticamente ---
def create_input_folder():
    input_path = "/content/SmartStitch/input"
    os.makedirs(input_path, exist_ok=True)
    return input_path

# --- pasta imutavel ---
input_folder = "input"

split_height = widgets.IntText(
    value=5000,
    description="Split H:",
    layout=widgets.Layout(width="40%")
)

output_format = widgets.Dropdown(
    options=[".png", ".jpg", ".webp", ".bmp", ".psd", ".tiff", ".tga"],
    value=".png",
    description="Formato:"
)

custom_width = widgets.IntText(
    value=0,
    description="Largura:",
    layout=widgets.Layout(width="30%")
)

detection_type = widgets.Dropdown(
    options=["pixel", "none"],
    value="pixel",
    description="Detecção:"
)

sensitivity = widgets.IntSlider(
    value=90, min=0, max=100, step=1,
    description="Sensib:"
)

quality = widgets.IntSlider(
    value=100, min=1, max=100, step=1,
    description="Qualidade:"
)

ignorable_pixels = widgets.IntText(
    value=5,
    description="Ign Pixels:"
)

scan_line = widgets.IntSlider(
    value=5, min=1, max=100, step=1,
    description="Scan Line:"
)

# --- Checkbox para zip e download ---
zip_download = widgets.Checkbox(
    value=True,
    description="Zipar e Baixar",
    indent=False
)

# --- 2. Saída e Função de Upload ---
output = widgets.Output()

def upload_images(b):
    output.clear_output()
    with output:
        input_path = '/content/SmartStitch/input'
        print(f"📁 Usando pasta input: {input_path}")

        print("📤 Selecione as imagens para upload...")

        original_cwd = os.getcwd()
        try:
            os.chdir(input_path)
            uploaded = files.upload()
        finally:
            os.chdir(original_cwd)

        if uploaded:
            print(f"📥 {len(uploaded)} arquivo(s) salvo(s) diretamente na pasta input...")
            for filename in uploaded.keys():
                dst_path = f"{input_path}/{filename}"
                print(f"✅ {filename} → {dst_path}")

            print(f"🎉 Upload concluído! {len(uploaded)} imagem(s) salvas em {input_path}")
        else:
            print("❌ Nenhum arquivo foi selecionado")

def run_smartstitch(b):
    output.clear_output()

    # Garantir que a pasta existe antes de executar
    input_path = create_input_folder()

    with output:
        # Verificar se há imagens na pasta
        if not os.path.exists(input_path):
            print("❌ Pasta input não existe!")
            return

        images = [f for f in os.listdir(input_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp', '.tiff', '.tga', '.psd'))]
        if not images:
            print("❌ Nenhuma imagem encontrada na pasta input!")
            print("💡 Use o botão 'Upload' para enviar imagens primeiro.")
            return

        print(f"📸 Encontradas {len(images)} imagem(s): {', '.join(images[:3])}{'...' if len(images) > 3 else ''}")

    cmd = [
        "python", "SmartStitchConsole.py",
        "-i", input_folder,
        "-sh", str(split_height.value),
        "-t", output_format.value
    ]

    if custom_width.value > 0:
        cmd += ["-cw", str(custom_width.value)]
    if detection_type.value:
        cmd += ["-dt", detection_type.value]
    if sensitivity.value:
        cmd += ["-s", str(sensitivity.value)]
    if quality.value:
        cmd += ["-lq", str(quality.value)]
    if ignorable_pixels.value:
        cmd += ["-ip", str(ignorable_pixels.value)]
    if scan_line.value:
        cmd += ["-sl", str(scan_line.value)]

    with output:
        print(f"\n🚀 Executando: {' '.join(cmd)}")
        print("-" * 50)
        process = subprocess.run(cmd, text=True, capture_output=True)

        if process.stdout.strip():
            print("📋 Output:")
            print(process.stdout)

        if process.stderr.strip():
            print("⚠️ Avisos/Erros:")
            print(process.stderr)

        print("-" * 50)
        print("✅ Processamento concluído!")

        # Zip e download se checkbox ativado
        if zip_download.value:
            output_folder = "/content/SmartStitch/input [stitched]"
            if os.path.exists(output_folder):
                zip_path = "/content/SmartStitch/stitched_results"
                shutil.make_archive(zip_path, 'zip', output_folder)
                zip_file = f"{zip_path}.zip"
                print(f"📦 Zip criado: {zip_file}")
                print("⬇️ Iniciando download...")
                files.download(zip_file)
            else:
                print(f"❌ Pasta de saída não encontrada: {output_folder}")

def clear_input_folder(b):
    output.clear_output()
    input_path = '/content/SmartStitch/input'
    with output:
        if os.path.exists(input_path):
            for filename in os.listdir(input_path):
                file_path = os.path.join(input_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"❌ Erro ao limpar {file_path}: {e}")
            print("✅ Pasta de entrada limpa!")
        else:
            print("❌ Pasta de entrada não existe!")

def clear_output_folder(b):
    output.clear_output()
    output_folder = '/content/SmartStitch/input [stitched]'
    with output:
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)
            print("✅ Pasta de saída limpa!")
        else:
            print("❌ Pasta de saída não existe!")

# --- 3. Botões ---
upload_button = widgets.Button(
    description="📤 Upload",
    button_style="info",
    tooltip="Enviar imagens para a pasta input"
)
upload_button.on_click(upload_images)

run_button = widgets.Button(
    description="▶️ Executar",
    button_style="success",
    tooltip="Executar SmartStitch com as configurações atuais"
)
run_button.on_click(run_smartstitch)

clear_input_button = widgets.Button(
    description="Lpar Entrada",
    button_style="danger",
    tooltip="Limpar a pasta de entrada"
)
clear_input_button.on_click(clear_input_folder)

clear_output_button = widgets.Button(
    description="Lpar Saida",
    button_style="danger",
    tooltip="Limpar a pasta de saida"
)
clear_output_button.on_click(clear_output_folder)

# --- 4. Montar Layout ---
ui = widgets.VBox([
    widgets.HTML("<h3>🧵 SmartStitch Console GUI</h3>"),
    widgets.HTML("<h4>⚙️ Configurações</h4>"),
    widgets.HBox([split_height, custom_width]),
    widgets.HBox([output_format, detection_type]),

    widgets.HTML("<h4>🎚️ Parâmetros Avançados</h4>"),
    sensitivity,
    quality,
    ignorable_pixels,
    scan_line,

    widgets.HTML("<h4>🎬 Execução</h4>"),
    widgets.HBox([upload_button, run_button, clear_input_button, clear_output_button]),
    zip_download,

    widgets.HTML("<h4>📊 Output</h4>"),
    output
])

# Criar a pasta input automaticamente ao inicializar
print("🔧 Inicializando SmartStitch GUI...")
create_input_folder()
#print("✅ Pasta input criada em /content/SmartStitch/input")

display(ui)